In [124]:
import pandas as pd
import os
import glob
import re

In [177]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/Daily Flood Report 1 Aug 2023 - 12 Oct 2023'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned'

In [42]:
# List of all Excel sheets shared in the dump
excel_files = []
for foldername, subfolders, filenames in os.walk(data_path):
    for filename in filenames:
        if filename.endswith(".xlsx"):
            excel_files.append(os.path.join(foldername, filename))

# Population and Crop Area

In [15]:
df = pd.read_excel(excel_files[11])

In [56]:
# Starting and Ending row index of the Population nd Crop Area Affected section
start = df[df.iloc[:,0].str.contains("Population").fillna(False)].index[0]

for index,row in df[start+1:].fillna('').iterrows():
    if row[0]=='':
        continue
    else:
        end = index
        break
        

In [67]:
# Population and Crop Area Affected section DataFrame
population_crop_df = df[start+1:end-1].reset_index(drop=True) #Removing Total row
population_crop_df.columns = df.iloc[start,:].to_list()
population_crop_df

,Population And Crop Area Affected,District/Sub-division*,Male,Female,Children,Total Population,Population Details,NaN,Crop Area ( in Hect. ),Crop Area Details,NaN
0,NaN,Dhemaji,132,114,62,308,"( Dhemaji | 260 ), ( Jonai | 48 )",NaN,37.33,"( Dhemaji | 16.00 ), ( Jonai | 21.33 )",NaN
1,NaN,Dibrugarh,75,72,59,206,( Dibrugarh East | 206 ),NaN,0,( Dibrugarh East | 0.00 ),NaN


In [198]:
# Loop across all excels
population_crop_dfs = []
for excel_file in excel_files:
     df = pd.read_excel(excel_file)
    # Date of excel
    date = re.findall('\d{2}\.\d{2}\.\d{4}',excel_file)[0]

   
    # Starting and Ending row index of the Population nd Crop Area Affected section
    start = df[df.iloc[:,0].str.contains("Population").fillna(False)].index[0]

    for index,row in df[start+1:].fillna('').iterrows():
        if row[0]=='':
            continue
        else:
            end = index
            break

    # Population and Crop Area Affected section DataFrame
    population_crop_df = df[start+1:end-1].reset_index(drop=True) #Removing Total row
    population_crop_df.columns = df.iloc[start,:].to_list()
    
    if population_crop_df.shape[0]==0:
        continue
    else:
        population_crop_df['date'] = date
        population_crop_dfs.append(population_crop_df.dropna(axis=1)) #There are extra columns with null values at times

In [194]:
master_population_crop_df = pd.concat(population_crop_dfs).reset_index(drop=True)
master_population_crop_df

,District/Sub-division*,Male,Female,Children,Total Population,Population Details,Crop Area ( in Hect. ),Crop Area Details,date
0,Dhemaji,1159,1007,587,2753,"( Sissiborgaon | 1428 ), ( Dhemaji | 301 ), ( ...",748.91,"( Sissiborgaon | 25.50 ), ( Dhemaji | 14.00 ),...",16.08.2023
1,Dibrugarh,109,93,8,210,"( Dibrugarh West | 0 ), ( Chabua | 0 ), ( Dibr...",978,"( Dibrugarh West | 717.00 ), ( Chabua | 261.00...",16.08.2023
2,Golaghat,0,0,0,0,( Bokakhat | 0 ),75,( Bokakhat | 75.00 ),16.08.2023
3,Sivasagar,4120,5261,1509,10890,"( Nazira | 505 ), ( Amguri | 0 ), ( Sivsagar |...",1717,"( Nazira | 0.00 ), ( Amguri | 42.00 ), ( Sivsa...",16.08.2023
4,Sonitpur,0,0,0,0,( Tezpur | 0 ),18,( Tezpur | 18.00 ),16.08.2023
...,...,...,...,...,...,...,...,...,...
276,Dhemaji,508,500,412,1420,"( Dhemaji | 207 ), ( Gogamukh | 1213 )",28,"( Dhemaji | 8.00 ), ( Gogamukh | 20.00 )",06.10.2023
277,Sivasagar,501,546,246,1293,"( Amguri | 47 ), ( Nazira | 1246 )",167.4,"( Amguri | 51.40 ), ( Nazira | 116.00 )",06.10.2023
278,Dhemaji,512,504,414,1430,"( Dhemaji | 217 ), ( Gogamukh | 1213 )",28,"( Dhemaji | 8.00 ), ( Gogamukh | 20.00 )",05.10.2023
279,Dhemaji,79,77,49,205,( Dhemaji | 205 ),9,( Dhemaji | 9.00 ),09.10.2023


In [195]:
#Disaggregating data for each revenue circle
rev_circles1=[]
populations=[]
dates1=[]
districts1 = []

rev_circles2=[]
crop_areas=[]
dates2=[]
districts2 = []


for idx, row in master_population_crop_df.iterrows():
    population_details = row['Population Details']
    crop_details = row['Crop Area Details']
    date = row['date']
    district = row['District/Sub-division*']
    
    
    for population_detail in population_details.split(','):
        rev_circle = re.findall(r'[A-Za-z]+', population_detail)[0]
        rev_circles1.append(rev_circle)
        
        population = re.findall('\d+', population_detail)[0]
        populations.append(population)
        
        dates1.append(date)
        districts1.append(district)
        
    
    
    for crop_detail in crop_details.split(','):
        rev_circle = re.findall('[A-Za-z]+', crop_detail)[0]
        rev_circles2.append(rev_circle)
        
        crop_area = re.findall('\d+', crop_detail)[0]
        crop_areas.append(crop_area)
        
        dates2.append(date)
        districts2.append(district)

In [196]:
master_population_disaggregated_df = pd.DataFrame([dates1, districts1, rev_circles1, populations]).T
master_population_disaggregated_df.columns = ['district', 'revenue_ci', 'date','population_affected']
master_population_disaggregated_df.to_csv(cleaned_data_path+'/population_affected_2023_aug_sep.csv', index = False)

In [197]:
master_croparea_disaggregated_df = pd.DataFrame([dates2, districts2, rev_circles2, crop_areas]).T
master_croparea_disaggregated_df.columns = ['district', 'revenue_ci', 'date','crop_area_affected']
master_croparea_disaggregated_df.to_csv(cleaned_data_path+'/crop_area_affected_2023_aug_sep.csv', index = False)

# INFRA - ROAD

In [ ]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
        if other_damages == True:
            NO_DAMAGES = len(re.findall('\|', SUB_DETAIL))
            print(NO_DAMAGES)
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES-1
            continue 
            
        NO_DAMAGES_DateAlgo = len(re.findall('2023',SUB_DETAIL)) + len(re.findall(r'\d+/\d+/\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+-\d+-\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+\.\d+\.\d{2}\s+', SUB_DETAIL))
        # Number of 2022 in the DETAIL +  Number of dd/mm/22 in the DETAIL +  Number of dd-mm-22 in the DETAIL + Number of dd.mm.22 in the DETAIL
        NO_DAMAGES_LongAlgo = len(re.findall('Long -',SUB_DETAIL)) 
    
        if float(ROW['Number of Damages (Long)'])==float(ROW['Number']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        elif float(ROW['Number of Damages (Long)'])>float(ROW['Number of Damages (Date)']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        else:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_DateAlgo
    
    
    return REV_CIRCLE_DAMAGES_DICT    

In [233]:
df = pd.read_excel(excel_files[0])

In [234]:
# Starting and Ending row index of the Population nd Crop Area Affected section
start = df[df.iloc[:,0].str.contains("Road").fillna(False)].index[0]

for index,row in df[start+1:].fillna('').iterrows():
    if row[0]=='':
        continue
    else:
        end = index
        break
        

In [235]:
# Roads damaged section DataFrame
roads_df = df[start+1:end-1].reset_index(drop=True) #Removing Total row
roads_df.columns = df.iloc[start,:].to_list()
roads_df

,Infrastructure Damaged - Road,District/Sub-division*,Number,Details,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Dhemaji,1,Dhemaji - Singimari to Jiadhal Chariali | Sing...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:
roads_df.shape

(1, 11)

In [248]:
# Loop across excels
roads_dfs = []
for excel_file in excel_files:
    df = pd.read_excel(excel_file)
    
    # Date of excel
    date = re.findall('\d{2}\.\d{2}\.\d{4}',excel_file)[0]
    
    # Starting and Ending row index of the Population nd Crop Area Affected section
    start = df[df.iloc[:,0].str.contains("Road").fillna(False)].index[0]
    for index,row in df[start+1:].fillna('').iterrows():
        if row[0]=='':
            continue
        else:
            end = index
            break
            
    # Roads damaged section DataFrame
    roads_df = df[start+1:end-1].reset_index(drop=True) #Removing Total row
    roads_df.columns = df.iloc[start,:].to_list()
        
    if roads_df.shape[0]==0:
        continue
    else:
        roads_df['date'] = date
        roads_dfs.append(roads_df.dropna(axis=1)) #There are extra columns with null values at times

In [249]:
master_roads_df = pd.concat(roads_dfs).reset_index(drop=True)
master_roads_df

,District/Sub-division*,Number,Details,date
0,Dhemaji,1,Dhemaji - Singimari to Jiadhal Chariali | Sing...,16.08.2023
1,Sivasagar,1,Amguri - Hiloidhari gaon sashan approach road....,08.08.2023
2,Chirang,1,Bijni - Ballamguri to Rangijhora via Khungkraj...,14.08.2023
3,Sonitpur,2,Chariduar - NH-15 TO GUDAMGHAT MIRI CONNECTING...,14.08.2023
4,Bajali,1,Bajali - Pub Barala to Golibandha Road | Pub B...,29.08.2023
...,...,...,...,...
85,Bongaigaon,2,Boitamari - Kabaitary abhayapuri road to mohan...,22.09.2023
86,Jorhat,1,Titabor - Gar Ali Road at 27th KM | Bekajan Ch...,22.09.2023
87,Bongaigaon,1,Srijangram - Nayagaon Sivmandir Road | Nayagao...,13.09.2023
88,Dhemaji,2,Gogamukh - Baligaon to Bordoibam Road. | From ...,11.09.2023


In [252]:
master_roads_df['Details'][11]

'Chariduar - SARUPATGAON VILLAGE ROAD | SARUPATGAON VILLAGE | SARUPATGAON VILLAGE ROAD IS SUBMERGED IN DIFFERENT STRETCHES BY FLOOD WATER OF RIVER JARASHAR LIKELY FROM CH.680.00 M TO CH 900.00M , CH 1200.00 M TO 1270 M AND CH 2080 M TO 2185 M REPORTED ON DTD 29.08.2023 | (Long - 92.782846, Lat - 26.780090, Dept - PWD (Roads)), \nBAMGAON 10 NO LINE CONNECTING ROAD | BAMGAON 10 NO LINE | BAMGAON 10 NO LINE CONNECTING ROAD IS SUBMERGED IN DIFFERENT STRETCHES LIKELY FROM CH 420 .00 M TO 900.00 M , CH 1300 M TO CH.1500 .00 M AND CH 1800.00 M TO 1900.00 M REPORTED ON DTD 29/08/2023 | (Long - 92.799962, Lat - 26.806250, Dept - PWD (Roads)), \nBALIPARA KHELMATI TO AMARIBARI ROAD | BALIPARA KHELMATI | BALIPARA KHELMATI TO AMARIBARI ROAD  DAMAGED BADLY BY INCESSANT RAIN IN NEAR BY AREA AT 3RD KM . REPORTED ON DTD 29/08/2023 | (Long - 92.755527, Lat - 26.812091, Dept - PWD (Roads)), \nNEW BAITHABHANGA CENTRE TO BAMUNGOLA MANSIRI CONNECTING ROAD | NEW BAITHABHANGA CENTRE | NEW BAITHABHANGA CENTRE 